### Load the COCO sentences and convert into a form compatible as input to myTupleExtractor

In [4]:
import json
from collections import defaultdict
import random 
import os

# set seed for random, so that every run gives the same val-test split
random.seed(32)
data = json.loads(open('data/simplesentences_coco.json', 'r').read())
dataset = defaultdict(list)

for x in data:
    dataset[x[1][0][0]].append(x[0][0])

# name the val and test files
val_file = 'cs_val.json'
test_file = 'cs_test.json'

# put half the images in val and half in test
all_images = dataset.keys()

val_id = random.sample(xrange(len(all_images)), len(all_images)/2)
test_id = [x for x in xrange(len(all_images)) if x not in val_id]

val_data = {all_images[k]: dataset[all_images[k]] for k in val_id} 
test_data = {all_images[k]: dataset[all_images[k]] for k in test_id}
sent_val = [sent for key in val_data for sent in val_data[key]]
sent_test = [sent for key in test_data for sent in test_data[key]]

print len(sent_val), len(sent_test)
print len(val_data), len(test_data)
#json.dump(val_data, open(os.path.join('data', val_file),'w'))
#json.dump(test_data, open(os.path.join('data', test_file), 'w'))


24790 24797
4956 4957


### Run tuple extraction on val set

In [22]:
run myTupleExtractor.py data/cs_val.json


data/cs_val.json

Dataset has 24790 sentences
Running Part-Of-Speech (POS) tagging
Running Reverb Tuple Extraction
Total number of relations is 1315 and total number of nouns is 3127


In [23]:
import operator
import json
from collections import defaultdict
from pandas import *

sorted_x = sorted(count_rels.items(), key=operator.itemgetter(1), reverse=True)

val_tuples = []
for y in sorted_x:
    if y[1]>=4:
        val_tuples.append(y[0])
        
all_val_tuples = [tup for key in tuples for tup in tuples[key]]

filtered_val_tuples = []

val_by_rel = defaultdict(list)
for t in all_val_tuples:
    if t[1] in val_tuples and t[1]!='':
        val_by_rel[t[1]].append(t)
        filtered_val_tuples.append(t)
        
fil_val_nouns = []
for t in filtered_val_tuples:
    fil_val_nouns.append(t[0])
    fil_val_nouns.append(t[1])

#json.dump(val_by_rel, open('val_by_rel.json', 'w'))

# make a table with some 10 examples for each relation
#print_val_rels = defaultdict(list)
#for rel in val_by_rel:
#    show_ex = val_by_rel[rel][0:4]
#    print_val_rels[rel] = show_ex

#D = DataFrame(print_val_rels).transpose()

#D

### Run tuple extraction on test set

In [24]:
run myTupleExtractor.py data/cs_test.json


data/cs_test.json

Dataset has 24797 sentences
Running Part-Of-Speech (POS) tagging
Running Reverb Tuple Extraction
Total number of relations is 1245 and total number of nouns is 3034


In [26]:
import operator
from collections import defaultdict
test_by_rel = defaultdict(list)

sorted_x = sorted(count_rels.items(), key=operator.itemgetter(1), reverse=True)

all_test_tuples = [(i,tup) for i,key in enumerate(tuples) for tup in tuples[key]]


test_tuples = []
for y in sorted_x:
    if y[1]>=4:
        test_tuples.append(y[0])
    
filtered_test_tuples = []
for i, t in all_test_tuples:
    if t[1] in test_tuples and t[1]!='':
        test_by_rel[t[1]].append((i,t))
        filtered_test_tuples.append(t)

fil_test_nouns = []
for t in filtered_test_tuples:
    fil_test_nouns.append(t[0])
    fil_test_nouns.append(t[1])
print len(set(fil_test_nouns))
print len(set(fil_val_nouns))
print len(filtered_val_tuples)
print len(filtered_test_tuples)

1305
1321
4851
4780


In [31]:
print len(filtered_test_tuples), len(filtered_val_tuples)

print len(filtered_nouns_test), len(filtered_nouns_val)

213 

4779 4851
2350 1290


### Check how many relations occur with different P and S in test than in val

In [56]:
from collections import defaultdict
import pdb
import numpy as np

common = defaultdict(float)
for rel in val_by_rel:
    if rel in test_by_rel.keys():
        val_tuples = val_by_rel[rel]
        test_tuples = test_by_rel[rel]
        
        count = 0
        for t in val_tuples:
            if t in test_tuples:
                 count += 1
        common[rel] = count/float(len(val_tuples))

np.mean(np.array(common.values()))

0.13338169846750486

### Tuples which are in val but not in test